In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os
import bcolz
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import time
import math
import pickle
from tqdm import tqdm
from torch.autograd import Variable
# plotting
import matplotlib
matplotlib.use('Agg')
from vad_utils import batchData
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
import json

In [2]:
model_path = "models/20190412 18-08-14"
param_path = os.path.join(model_path,"model_parameters.json")
with open(param_path) as json_file:  
    params = json.load(json_file)
print(params)

{'hiddenSize': 350, 'latentSize': 300, 'batchSize': 32, 'iterations': 20, 'learningRate': 0.0001, 'gradientClip': 1, 'useBOW': True, 'bidirectionalEncoder': True, 'reduction': 4, 'device': 'cuda', 'useLatent': True}


In [3]:

"""
Set seed #
"""
seed = 1337

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

def loadDataset(path = '../Datasets/Reviews/dataset_ready.pkl'):
    return pickle.load(open(path, 'rb'))

In [4]:
print("Loading parameters..", end=" ")
batchSize  = 32
iterations = 1
bidirectionalEncoder = params['bidirectionalEncoder']
# device = "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Done.")

print("Loading dataset..", end=" ")
dataset = loadDataset()
# setup store parameters
id2word = dataset['id2word']
word2id = dataset['word2id']
weightMatrix = dataset['weights']
train = dataset['train']
validation = dataset['validation']
cutoff = dataset['cutoff']
paddingID = word2id['<pad>']
print("Done.")

print("Converting dataset weights into tensors..", end=" ")
# convert dataset into tensors
weightMatrix = torch.tensor(weightMatrix, dtype=torch.float)
print("Done.")

# batching data
print("Batching Data..",end=" ")

random.shuffle(validation)

trainx = [x[0] for x in train]
trainy = [x[1] for x in train]
valx = [x[0] for x in validation]
valy = [x[1] for x in validation]

trainx = batchData(trainx, paddingID, device, batchSize, cutoff)
trainy = batchData(trainy, paddingID, device, batchSize, cutoff)
valx = batchData(valx, paddingID, device, batchSize, cutoff)
valy = batchData(valy, paddingID, device, batchSize, cutoff)

traindata = (trainx, trainy)
valdata = (valx, valy)
print("Done.")

# setup variables for model components initialisation
maxReviewLength = cutoff
vocabularySize = len(id2word)
embeddingDim = weightMatrix.shape[1]

embedding_shape = weightMatrix.shape

# setup additional variables
hiddenSize = params['hiddenSize']
latentSize = params['latentSize']
# setup embeddiing

embedding = nn.Embedding(
        num_embeddings=vocabularySize,
        embedding_dim=embeddingDim,
        padding_idx=paddingID,
    _weight=weightMatrix
    ).to(device)

Loading parameters.. Done.
Loading dataset.. Done.
Converting dataset weights into tensors.. Done.
Batching Data.. Done.


In [5]:
from vad import Encoder, Decoder, loss_function
print("Initialising model components..", end=" ")
modelEncoder = Encoder(embedding, vocabularySize,
                       paddingID, hiddenSize, bidirectionalEncoder).to(device)

modelDecoder = Decoder(embedding, vocabularySize,
                       paddingID, batchSize, maxReviewLength, hiddenSize, latentSize, bidirectionalEncoder).to(device)

criterion = nn.NLLLoss(ignore_index=paddingID)


print("Done.")

# load models
print("Loading model weights..", end=" ")
modelEncoder.load_state_dict(torch.load(os.path.join(model_path,'encoder.pth')))
modelDecoder.load_state_dict(torch.load(os.path.join(model_path,'decoder.pth')))
print("Done.")

Initialising model components.. Done.
Loading model weights.. Done.


In [6]:
def evaluate(x,
             xLength,
             y,
             yLength,
             encoder,
             decoder,
             device,
             criterion,
             word2id
            ):
    
    loss = 0
    
    # initalise input and target lengths
    inputLength = x[0].size(0)
    targetLength = y[0].size(0)
    batchSize = x.shape[0]

    # set up encoder computation
    encoderHidden = encoder.initHidden(batchSize).to(device)
    
    x = x.to(device)
    y = y.to(device)
    
    # set up encoder outputs
    encoderOutputs, encoderHidden = encoder(x, encoderHidden, xLength)

    # set up the variables for decoder computation
    decoderInput = torch.tensor([word2id["<sos>"]] * batchSize, dtype=torch.long, device=device)
    
    decoderHidden = encoderHidden[-1]
    decoderOutput = None
    decoderOutputs = []
    
    # Run through the decoder one step at a time. This seems to be common practice across
    # all of the seq2seq based implementations I've come across on GitHub.
    for t in range(yLength[0]):
        # compute the output of each decoder state
        decoderOutput, decoderHidden = decoder(decoderInput, encoderOutputs, xLength, decoderHidden, device, back=None)

        decoderOutputs.append(decoderOutput)

        decoderInput = decoderOutput.argmax(1)
        decoderHidden = decoderHidden.squeeze(0)
        
    return decoderOutputs, loss

In [7]:
# from train_vad import evalVAD as evaluate
def evaluateModel(batched_data,
                   encoder,
                   decoder,
                   criterion,
                   id2word,
                  ):
    
    encoder.eval()
    decoder.eval()
    numbatches = len(batched_data)
    for batch in tqdm(range(0,3)):
        # each batch is composed of the 
        # reviews, and a sentence length.
        x, xLength = batched_data[0][batch][0], batched_data[0][batch][1]
        y, yLength = batched_data[1][batch][0], batched_data[1][batch][1]
        
        outputs, losses = evaluate(x,
                                   xLength,
                                   y,
                                   yLength,
                                   encoder,
                                   decoder,
                                   device,
                                   criterion,
                                   word2id)
        break
    return x, y, outputs

In [8]:
x, y, outputs = evaluateModel(traindata,
                   modelEncoder,
                   modelDecoder,
                   criterion,
                   id2word
                  )

  0%|          | 0/3 [00:00<?, ?it/s]


In [9]:
def convertRealID2Word(id2word, y):
    entries = [[id2word[x.item()] for x in y[entry].cpu()] for entry in range(len(y))]
    for i in range(len(entries)):
        entries[i] = " ".join(entries[i])
        entries[i] = entries[i].replace("<pad> ", "")
#         entries[i] = entries[i][43:]
    return entries
        
def convertDecoderID2Word(id2word, outputs):
    entries = []
    for batch_line in outputs:
        entry = [torch.argmax(batch_line[i]).cpu().item() for i in range(len(batch_line))]
        entries.append([id2word[i] for i in entry])
     
    words = []
    for i in range(len(outputs[0])):
        tokens = [entries[j][i] for j in range(len(entries))]
        # find the eos token
        try:
            tokenpos = tokens.index("<eos>")
        except:
            tokenpos = len(tokens)
        # remove extra eos tokens and padding values if they exist.
        words.append(" ".join(tokens[:tokenpos+1]))
    return words

In [10]:
before = convertRealID2Word(id2word,x)
reference = convertRealID2Word(id2word,y)
words = convertDecoderID2Word(id2word,outputs)

for i in range(len(reference)):
    print("INPUT:", before[i])
    print("REFERENCE:", reference[i])
    print("MODEL:    ",words[i])
    print()

INPUT: b 0 0 0 0 0 j 1 u b rating_5.0 polarity_0.0 <unk> he wanted to use one of them with his new pc that has usb inputs only ( i had previously bought him a ps/2 keyboard to usb adapter so he could use his * other * beloved , only slightly newer , keyboard . ) <eos> <pad>
REFERENCE: <unk> he wanted to use one of them with his new pc that has usb inputs only ( i had previously bought him a ps/2 keyboard to usb adapter so he could use his * other * beloved , only slightly newer , keyboard . ) <eos> <pad>
MODEL:     <unk> 'm a happy with this product . <eos>

INPUT: b 0 0 0 0 0 j 1 u b rating_5.0 polarity_0.0 <unk> he called me asking if i could find a way to get his old " big round plug " keyboard with the new <unk> belkin <unk> adapter was just what he needed ( combined with the aforementioned <unk> adapter ) . <eos> <pad>
REFERENCE: <unk> i had it sent to him and he called me a few days later , with delight , saying that the adapter worked perfectly , adding that he was astonished an

In [11]:
# deletes models with no weights saved to them.
import shutil
import os
for x in os.listdir('models'):
    subpath = os.path.join('models', x)
    if "encoder.pth" not in os.listdir(subpath):
        print("bad", subpath, os.listdir(subpath))
        shutil.rmtree(subpath)